In [1]:
from pymongo import MongoClient
from pprint import pprint


In [2]:
from bs4 import BeautifulSoup as bs
import requests
import json


## Готовим наш парсер и данные для их дальнейшего сохранения в бд

In [20]:
# Используем класс, чтобы уже структурно хранить инфу. Также добавил словарь айдишников для разделения хх и с-дж
class Vacancies():
    def __init__(self,vacancy = 'python'):
        self.vacancy = vacancy
        self.id_keys = {
            'hh':'0',
            'sj' :'1'
        }
        print('You can parse HH and Super Job. Try .hh_Parse() or sj_parse(). To get id keys use showIDkeys()')
    def hh_Parse(self):
        page_num = 1
        base_link='https://hh.ru'

        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
        header = {
            'User-Agent':user_agent
        }
        vacancies_hh = []
        id_hh = set()
        while True:
            main_link = f'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={self.vacancy}&page={str(page_num)}'
            response= requests.get(main_link,headers = header).text
            soup = bs(response,'lxml')
            try:
                next_link_part = soup.find_all('a',{'class':'HH-Pager-Controls-Next'})[0]['href']
                next_link = base_link+next_link_part
            except:
                break

            if next_link ==f'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={self.vacancy}&page={str(page_num+1)}':

               # response= requests.get(main_link,headers = header).text
                #soup = bs(response,'lxml')
                vacancy_block = soup.find_all('div',{'class':'vacancy-serp'})[0]
                vacancy_all = vacancy_block.find_all('div',{'class':'vacancy-serp-item'
                })
                for vac in vacancy_all:
                    vacancy_data = {}
                    vacancy_name = vac.find_all('div',{'class':'vacancy-serp-item__info'})[0].find('a').getText()
                    vacancy_link = vac.find_all('div',{'class':'vacancy-serp-item__info'})[0].find('a')['href']
                    vacancy_id = self.id_keys['hh'] + vacancy_link.rsplit('?',1)[0].rsplit('/',1)[1]
                    if '-' in vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText():
                        vacancy_money_min, vacancy_money_max = vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText().split('-')
                        vacancy_money_max, currency = vacancy_money_max.rsplit(' ',1)
                        currency = currency.rstrip('.')  
                        vacancy_data['vacancy_money_min'] = int(vacancy_money_min.replace('\xa0', ''))
                        vacancy_data['vacancy_money_max'] = int(vacancy_money_max.replace('\xa0', ''))
                        vacancy_data['currency'] = currency
                        vacancy_data['from_to'] = 'both'
                    elif '0' in vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText():
                        vacancy_money,currency = vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText().rsplit(' ',1)
                        from_to,vacancy_money = vacancy_money.replace('\xa0', ' ').split(' ',1)
                        currency = currency.rstrip('.')
                        if from_to == 'от':
                            vacancy_data['vacancy_money_min'] = int(vacancy_money.replace(' ', ''))
                            vacancy_data['currency'] = currency
                            vacancy_data['from_to'] = from_to
                            vacancy_data['vacancy_money_max'] = None
                        else:
                            vacancy_data['vacancy_money_max'] = int(vacancy_money.replace(' ', ''))
                            vacancy_data['vacancy_money_min'] = None
                            vacancy_data['currency'] = currency
                            vacancy_data['from_to'] = from_to
                    else:
                        vacancy_data['vacancy_money_min'] = None
                        vacancy_data['vacancy_money_max'] = None
                        vacancy_data['currency'] = None
                        vacancy_data['from_to'] = None
                    company = vac.find_all('div',{'class':'vacancy-serp-item__meta-info'})[0].getText().split('-')[0]
                    info = [vac.find_all('div',{'class':'g-user-content'})[e].getText() for e in range(len(vac.find_all('div',{'class':'g-user-content'})))]
                    source = 'hh'
                    vacancy_data['vacancy_name'] = vacancy_name
                    vacancy_data['vacancy_link'] = vacancy_link
                    vacancy_data['company'] = company.replace('\xa0', ' ')
                    vacancy_data['info'] = info
                    vacancy_data['source'] = source
                    vacancy_data['_id']=vacancy_id
                    vacancies_hh.append(vacancy_data)
                    id_hh.add(vacancy_id)
                page_num+=1
        self.id_hh = id_hh
        self.vacancies_hh = vacancies_hh
        return self.vacancies_hh,self.id_hh
    def sj_parse(self):
        page_num = 1
        base_link=f'https://www.superjob.ru'

        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
        header = {
            'User-Agent':user_agent
        }
        vacancies_SJ = []
        id_sj = set()
        count = 1
        while True:

            main_link = f'https://russia.superjob.ru/vacancy/search/?keywords={self.vacancy}&page={page_num}'
            page_num+=1
            response= requests.get(main_link,headers = header).text
            soup = bs(response,'lxml')
            try:
                vacancy_block = soup.find_all('div',{'class':'_1ID8B'})[0]
            except:
                break
            vacancy_all = vacancy_block.find_all('div',{'class':'QiY08 LvoDO'})

            for vac in vacancy_all:
                vacancy_data = {}
                vacancy_name = vac.find_all('a',{'class':'icMQ_'})[0].getText()
                vacancy_link = base_link+vac.find_all('a',{'class':'icMQ_'})[0]['href']
                vacancy_id = self.id_keys['sj']+vacancy_link.rsplit('.',1)[0].rsplit('-',1)[1]
                if vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})  == []:
                    vacancy_data['vacancy_money_min']=None
                    vacancy_data['vacancy_money_max']=None
                    vacancy_data['currency'] = None
                    vacancy_data['from_to'] = None
                    vacancy_data['currency'] = None


                elif '—' in vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText():
                    vacancy_money_min, vacancy_money_max = vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText().split('—')
                    vacancy_money_max, currency = vacancy_money_max.replace('\xa0', ' ').rsplit(' ',1)
                    currency = currency.rstrip('.')  
                    vacancy_data['vacancy_money_min'] = int(vacancy_money_min.replace('\xa0', ''))
                    vacancy_data['vacancy_money_max'] = int(vacancy_money_max.replace(' ', ''))
                    vacancy_data['currency'] = currency
                    vacancy_data['from_to'] = 'both'


                elif '0' in vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText():
                    vacancy_money,currency = vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText().replace('\xa0', ' ').rsplit(' ',1)
                    from_to,vacancy_money = vacancy_money.replace('\xa0', ' ').split(' ',1)
                    currency = currency.rstrip('.')
                    if from_to=='от':
                        vacancy_data['vacancy_money_min'] = int(vacancy_money.replace(' ',''))
                        vacancy_data['vacancy_money_max'] = None
                        vacancy_data['from_to'] = 'от'
                    else:
                        vacancy_data['vacancy_money_min'] = None
                        vacancy_data['vacancy_money_max'] = int(vacancy_money.replace(' ',''))
                        vacancy_data['from_to'] = 'до'
                    vacancy_data['currency'] = currency

                else:
                    vacancy_data['vacancy_money_min'] = None
                    vacancy_data['vacancy_money_max'] = None
                    vacancy_data['currency'] = None
                    vacancy_data['from_to'] = None
                try:
                    company = vac.find_all('span',{'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})[0].getText()
                except:
                    company = None
                info = [vac.find_all('div',{'class':'_3cLIl _3C76h _10Aay _2_FIo _1tH7S'})[e].getText() for e in range(len(vac.find_all('div',{'class':'_3cLIl _3C76h _10Aay _2_FIo _1tH7S'})))]
                source = 'SJ'
                vacancy_data['vacancy_name'] = vacancy_name
                vacancy_data['vacancy_link'] = vacancy_link
                vacancy_data['company'] = company
                vacancy_data['info'] = info
                vacancy_data['source'] = source
                vacancy_data['_id']=vacancy_id
                id_sj.add(vacancy_id)
                vacancies_SJ.append(vacancy_data)
                
        self.vacancies_SJ = vacancies_SJ
        self.id_sj = id_sj
        return self.vacancies_SJ,self.id_sj
    def showIDKeys(self):
        return self.id_keys
    def total_Vac(self):
        sj_vacancies,id_sj = self.sj_parse()
        hh_vacancies,id_hh = self.hh_Parse()
        total_vacancies= sj_vacancies+hh_vacancies
        id_total = id_sj|id_hh
        return total_vacancies,id_total
        
        

In [76]:
Vacanciess = Vacancies('python')

You can parse HH and Super Job. Try .hh_Parse() or sj_parse(). To get id keys use showIDkeys()


In [77]:
total_vac,total_id = Vacanciess.total_Vac()

## Данные подготовили. Начинаем работать с БД

In [5]:
client = MongoClient('127.0.0.1',27017)

In [62]:
db = client['Job_Vacancies']
vacancies = db.vacancies
ids = db.ids

In [74]:
# Добавляет только новые вакансии с той базы, что я спарсил
def addData_All(total_vac):
    count = 0
    count2 = 0
    for e in total_vac:
        try:
            vacancies.insert_one(e)
            count2+=1
        except:
            count+=1
            continue
    print('Done', count,': Duplicates dropped','\n', count2, 'new values added')

In [78]:
addData_All(total_vac)

Done 1945 : Duplicates dropped 
 0 new values added


In [54]:
def searchVac_more(num):
    count = 1
    filtered_list = []
    for vac in vacancies.find({
        '$or':[
            {'$and':[{'$and':[{'from_to':{'$ne':'both'}},{'from_to':{'$ne':None}}]},{'$or':[{'vacancy_money_min':{'$gte':num}},{'vacancy_money_max':{'$gte':num}}]}]}
            ,

            {'$and':[{'from_to':{'$eq':'both'}},{'vacancy_money_min':{'$gte':num}}]}
        ]
    }):
        filtered_list.append(vac)
        count+=1
    print(f'Total vacancies of your criteria{count}')
    return filtered_list

In [58]:
searchVac_more(10000)[0]

Total vacancies of your criteria361


{'_id': '133742193',
 'vacancy_money_min': 110000,
 'vacancy_money_max': None,
 'from_to': 'от',
 'currency': 'руб',
 'vacancy_name': 'Ведущий разработчик программного обеспечения (senior Python developer)',
 'vacancy_link': 'https://www.superjob.ru/vakansii/veduschij-razrabotchik-programmnogo-obespecheniya-33742193.html',
 'company': 'Макрософт',
 'info': ['Разработка back-end features in Python. Создание / разработка функционала. Интеграция front-end компонентов в приложениеПрактический опыт работы непосредственно в разработке ПО в качестве одной из ролей (программист, системный аналитик, системный…'],
 'source': 'SJ'}